In [1]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 

In [28]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.8
Method: manual login


In [29]:
names = [
    "MICHAEL AGUS",
    "DOROTHY BAINTON",
    "CHARLES BALCH",
    "JEFFREY BALSER",
    "KEVIN CATT",
    "HENRY CHAMBERS",
    "RICHARD CLARK",
    "STEPHANIE EISENBARTH",
    "PENG JI",
    "PETER FECCI",
    "TIPPI MACKENZIE",
    "PURNA KASHYAP"
]

namesTest = [
    "MICHAEL AGUS",
    "PETER FECCI",
    "TIPPI MACKENZIE",
    "PURNA KASHYAP"]

#Link to grant variables description: https://docs.dimensions.ai/dsl/datasource-grants.html#grants
search_Var = "[abstract + active_year + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + category_sdg + category_uoa + concepts + concepts_scores + date_inserted + dimensions_url + end_date + foa_number + funder_countries + funder_org_acronym + funder_org_cities + funder_org_countries + funder_org_name + funder_org_states + funder_orgs + funders + funding_aud + funding_cad + funding_chf + funding_cny + funding_currency + funding_eur + funding_gbp + funding_jpy + funding_nzd + funding_org_acronym + funding_org_cities + funding_org_city + funding_org_name + funding_org_states + funding_schemes + funding_usd + grant_number + id + investigators + keywords + language + language_title + linkout + original_title + project_numbers + research_org_cities + research_org_countries + research_org_names + research_org_state_codes + research_orgs + researchers + score + start_date + start_year + title]"

In [30]:
#using researcher ids
def researcherIds(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    # print(res.json) #list of dictionaries
    listDict = res.json['researchers']
    allIds = []
    for idv in listDict:
        allIds.append(idv['id'])
    allIds = [f'"{item}"' for item in allIds]
    allIds = '[' + ', '.join(allIds) + ']'
    print(f'Total of {len(allIds)} ids for {name}: ')
    
    res2 = dsl.query_iterative(f"""search researchers where id in {allIds} return researchers[id+obsolete+redirect]""")
    listDict2 = res2.json['researchers']
    allWorkingIDs = set()
    for person in listDict2:
        if person['obsolete'] == 0: #Current Working Id(s)
            allWorkingIDs.add(person["id"])
        else:
            for ids in person['redirect']:
                allWorkingIDs.add(ids)
    print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
    print(allWorkingIDs)
    return allWorkingIDs
#testIDs = researcherIds("MICHAEL AGUS")

In [31]:
#using researcher ids to find grants
def findingGrants(ids):
    all_grants = {}
    if len(ids) >= 1:
        for i in ids:
            res = dsl.query_iterative(f"""search grants where researchers = "{i}" return grants{search_Var}""")
            number = res.json['_stats']['total_count']
            print(f'{i} id has {number} grants')
            listDict = res.json['grants']
            name_list = []
            for grant in listDict:
                name_list.append(grant)
            all_grants[i]= name_list
    
    else:
        print(f'+++++++++++There are no working ids+++++++++++')
    return all_grants
        
#findingGrants(testIDs)

In [32]:
#Finds grants based on active researcher ids

def idGrants(nameList):
    allGrants = {}
    for name in nameList:
        nameIds = researcherIds(name)
        indvGrants = findingGrants(nameIds)
        allGrants[name] = indvGrants
    return allGrants
idGrants(namesTest)

Starting iteration with limit=1000 skip=0 ...
0-23 / 23 (0.70s)
===
Records extracted: 23
Starting iteration with limit=1000 skip=0 ...


Total of 501 ids for MICHAEL AGUS: 


0-23 / 23 (0.27s)
===
Records extracted: 23
Starting iteration with limit=1000 skip=0 ...


++++++++ FINAL 1 Working ID(s) +++++++++++
{'ur.0611721335.42'}


0-6 / 6 (0.45s)
===
Records extracted: 6
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


...

0-16 / 16 (0.43s)
===
Records extracted: 16
Starting iteration with limit=1000 skip=0 ...


++++++++ FINAL 1 Working ID(s) +++++++++++
{'ur.01007613452.86'}


0-3 / 3 (4.11s)
===
Records extracted: 3
Warnings:  8


ur.01007613452.86 id has 3 grants


{'MICHAEL AGUS': {'ur.0611721335.42': [{'abstract': 'Stress Hydrocortisone In Pediatric Septic Shock (SHIPSS) Project Summary Sepsis represents the most common cause of childhood mortality worldwide. In the United States alone, 200 cases of pediatric sepsis are diagnosed each day, with an associated hospital mortality rate of 5-10% and health care expenditures now approaching $5 billion annually. Moreover, nearly 40% of children admitted to pediatric intensive care units (PICUs) for septic shock have not regained their baseline health-related quality of life one year following the sepsis event. During early resuscitation of the child with septic shock, in addition to antibiotics, volume replacement, and vasoactive-inotropic support, the most recent pediatric treatment guidelines advise the practitioner to consider adjunctive hydrocortisone therapy if the patient “is at risk of absolute adrenal insufficiency or adrenal pituitary axis failure”. However, the potential benefits and risks o

In [10]:
#search index queries
def grantSearch(namesTest):
    all_grants= {}
    for name in namesTest:
        res = dsl.query_iterative(f"""search grants in investigators for "{name}" return grants{search_Var}""")
        number = res.json['_stats']['total_count']
        print(f'{name} has {number} grants')
        listDict = res.json['grants']
        for indvgrant in listDict:
            print(indvgrant)
            print()

grantSearch(namesTest)

Starting iteration with limit=1000 skip=0 ...
0-6 / 6 (0.46s)
===
Records extracted: 6
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


MICHAEL AGUS has 6 grants
{'abstract': 'Stress Hydrocortisone In Pediatric Septic Shock (SHIPSS) Project Summary Sepsis represents the most common cause of childhood mortality worldwide. In the United States alone, 200 cases of pediatric sepsis are diagnosed each day, with an associated hospital mortality rate of 5-10% and health care expenditures now approaching $5 billion annually. Moreover, nearly 40% of children admitted to pediatric intensive care units (PICUs) for septic shock have not regained their baseline health-related quality of life one year following the sepsis event. During early resuscitation of the child with septic shock, in addition to antibiotics, volume replacement, and vasoactive-inotropic support, the most recent pediatric treatment guidelines advise the practitioner to consider adjunctive hydrocortisone therapy if the patient “is at risk of absolute adrenal insufficiency or adrenal pituitary axis failure”. However, the potential benefits and risks of this recomm

0-3 / 3 (4.78s)
===
Records extracted: 3
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


PETER FECCI has 3 grants
{'abstract': 'ABSTRACT Glioblastoma (GBM), the most common primary malignant brain tumor, is among the most lethal of cancers. It is also among the most immunosuppressive, impeding the success of immune-based therapies. The tumor has an especially devastating impact on T cells, which either vanish from the blood and lymphoid organs or persist in a state of pronounced dysfunction. While T cell dysfunction has been well characterized, their disappearance has not, instead remaining a mystery for decades. Our novel data reveal that the missing T cells are frequently found in large numbers in the bone marrow. This proves true in both patients and mice with GBM, each of which harbor a 3 to 5-fold expansion in bone marrow T cell counts. Such expansion contrasts starkly with observations of AIDS-level CD4 T cell counts in blood and gross contraction of other lymphoid organs, including the spleen. Sequestration in bone marrow is suggested as T cells transferred into GBM

0-12 / 12 (0.50s)
===
Records extracted: 12
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


TIPPI MACKENZIE has 12 grants
{'abstract': 'Lysosomal storage disorders (LSDs) are severe diseases arising from mutations in critical enzymes and collectively have an estimated incidence of 1 in 5,000 to 1 in 5,500 live births. Patients with LSDs are at increased risk of serious perinatal morbidity and mortality, with some not even surviving to birth. The current treatment for pediatric patients, enzyme replacement therapy (ERT), is limited by three aspects: the progressive development (sometimes in utero) of organ-specific manifestations, the development of anti-ERT antibodies, and the inability of ERT to cross the blood-brain barrier to address neurologic effects. Thus, there is an unmet medical need to develop more effective therapies for patients with LSDs, starting before birth. In a mouse model of mucopolysaccharidosis type 7 (MPS7), we showed that in utero ERT (IUERT) followed by postnatal ERT improved survival, crossed the blood-brain barrier, ameliorated disease, and induced t

0-4 / 4 (4.42s)
===
Records extracted: 4
Warnings:  8


PURNA KASHYAP has 4 grants
{'abstract': 'PROJECT SUMMARY/ABSTRACT Irritable bowel syndrome is a globally prevalent disorder (~11%) characterized by an alteration in stool form/frequency and abdominal pain one or more days per week. Abdominal pain in IBS, like other forms of visceral pain, is often diffuse and poorly localized, making it difficult to delineate the site of pathology, and as a result there are few therapeutic options. Gut microbial products have been shown to be important luminal signals for abdominal pain, but these studies have largely focused on the colon. We recently found that small intestinal microbial composition is associated with gastrointestinal (GI) symptoms like abdominal pain, but the mechanisms underlying the role of small intestinal microbiota/microbial products in the pathophysiology of abdominal pain remains a critical knowledge gap. To address this gap, we will elucidate the sensory innervation of the proximal small intestine and identify the cellular an

In [12]:
#exact name searchers
def grantSearch(namesList):
    all_grants= {}
    for name in namesList:
        res = dsl.query_iterative(f"""search grants where investigators = "{name}" return grants{search_Var}""")
        number = res.json['_stats']['total_count']
        print(f'{name} has {number} grants')
        listDict = res.json['grants']
        name_list = []
        for grant in listDict:
            #print(grant.keys())
            name_list.append(grant)
        all_grants[name]= name_list
    return all_grants



grantSearch(namesTest)

Starting iteration with limit=1000 skip=0 ...
0-2 / 2 (0.58s)
===
Records extracted: 2
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


MICHAEL AGUS has 2 grants


0-3 / 3 (4.64s)
===
Records extracted: 3
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


PETER FECCI has 3 grants


0-11 / 11 (0.88s)
===
Records extracted: 11
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


TIPPI MACKENZIE has 11 grants


===
Records extracted: 0
Warnings:  8


PURNA KASHYAP has 0 grants


{'MICHAEL AGUS': [{'abstract': 'Septic shock accounts for up to 8% of pediatric intensive care unit (PICU) admissions worldwide, carries significant morbidity and has a mortality rate between 5% and 40% depending on the geographic region in which it occurs. Mainstays of treatment involve antibiotics, fluids and medications to improve blood pressure. However, the high morbidity and mortality of pediatric septic shock have led physicians to consider corticosteroids for unstable patients since corticosteroids are inexpensive and widely available. However, although the use of corticosteroids for septic shock has been widely debated for over 40 years, there is no clear evidence for or against this practice. Some pediatric studies have reported an improvement in blood pressure and organ function with corticosteroids while other studies have suggested an increase in blood sugar and the inability to fight infection. We therefore plan to conduct a randomized controlled trial to determine if hyd

In [ ]:
#I see that what shows up for individuals id also shows up in index queries